# Proyecto Final Chatbot - Deep Learning - ChatBot

Un Chatbot por lo general usa un modelo de redes recurrentes (o sus análogos de LSTM o GRU) del tipo Seq-to-Seq para entrena un sistema automático que responda ciertas preguntas que se le hagan o sea capaz de responder e interactuar en conversaciones. En este sentido, es parecido a un modelo de traducción, con la diferencia que la salida no es el mismo texto en otro idioma, sino la respuesta natural para continuar en una conversación. 

Para entrenar este modelo, es posible usar los datos de conversaciones de películas recopilada por Danescu et al. en 2011. Estos datos se conocen como “Cornell Movie – Dialogs Corpus”. Los pude encontrar en el siguiente link: https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

La idea es entrenar un modelo de Chatbot que sea capaz sea responder e interactuar en conversaciones.

## Importar librerias

In [2]:
import pandas as pd
import numpy as np

#Data Set
from convokit import Corpus, download

## Cargar Datos

In [ ]:
# Descargar datos directamente de "https://convokit.cornell.edu/documentation/movie.html"
corpus = Corpus(filename=download("movie-corpus"))

No configuration file found at /home/maro/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem


In [ ]:
# Extraer utterances
utterances = list(corpus.iter_utterances())

# Convertir a DataFrame
df_utterances = pd.DataFrame([{
    "id": utt.id,
    "speaker": utt.speaker.id,
    "text": utt.text,
    "conversation_id": utt.conversation_id,
    "reply_to": utt.reply_to
} for utt in utterances])

# mostrar sample
df_utterances.head()

,id,speaker,text,conversation_id,reply_to
0,L1045,u0,They do not!,L1044,L1044
1,L1044,u2,They do to!,L1044,None
2,L985,u0,I hope so.,L984,L984
3,L984,u2,She okay?,L984,None
4,L925,u0,Let's go.,L924,L924
